<img src="../../strathclyde_banner.png" alt="University of Strathclyde" width="100%">

# OFDM Transceiver

Following on from OFDM Fundamentals, this notebook introduces the major components of an OFDM transceiver. This includes a demonstration of multipath channel estimation and the one-tap equaliser. 

## Aims 

* Review major steps in OFDM transmitter 
* Pass signal through multipath channel and add AWGN noise 
* Review major steps in OFDM receiver 
* Demonstrate channel estimation and the one-tap equaliser

##  Table of Contents 

* [1. Introduction](#introduction)
* [2. OFDM Transmitter](#ofdmtx)
   * [2.1 Training Symbol Generation](#training)
   * [2.2 OFDM Symbol Generation](#data)
* [3. Wireless Channel](#channel)
* [4. OFDM Receiver](#ofdmrx)
   * [4.1 Channel Estimation](#chanest)

## 1. Introduction <a class="anchor" id="introduction"></a>

In this notebook, an overview of the main steps in an OFDM transceiver are provided. On the transmit side, this includes symbol generation, IFFT modulation and Cyclic Prefix (CP) addition. The OFDM signal is then passed through a baseband channel model, including multipath channel filter and Additive White Gaussian Noise (AWGN). The receiver stages include CP removal, FFT demodulation, channel estimation and one-tap equalisation. In order to assess the quality of the received signal, we inspect the received constellations before and after equalisation and measure Error Vector Magnitude (EVM). 

We will also mention some important aspects of the OFDM transceiver which are not explicitly covered in this notebook, such as timing and and frequency synchronisation. However, these stages are included in the accompanying RFSoC OFDM demonstration.   

## 2. OFDM Transmitter <a class="anchor" id="ofdmtx"></a>

In this sections, we will demonstrate some of the main steps in an OFDM transmitter. Firstly, let's import the necessary libraries and helper functions:

In [46]:
# Import necessary libraries 
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import signal

# Import helper function
from helper_functions import symbol_gen, psd, \
frequency_plot, scatterplot, calculate_evm, awgn

[Figure 2.1](#fig-2.1) shows the stages of the OFDM transmitter that will be demonsrated in this notebook:

<a class="anchor" id="fig-2.1"></a>
<figure>
<img src="images/ofdm_tx_3 copy.svg" style="width: 1000px;"/> 
    <figcaption><b>Figure 2.1: OFDM Transmitter</b></figcaption>
</figure>    

This is very similar to the final OFDM transmitter diagram shown in the OFDM fundamentals notebook. However, we have now addded a sub-carrier mapping stage after symbol generation. This is necessary because in real OFDM systems, only particular sub-carriers are used to carry data. The remaining sub-carriers are used to carry pilots for phase tracking and channel estimation in the receiver and null sub-carriers to relax the requirements for anti-imaging and anti-aliasing filters in the transmitter and receiver respectively.  

In the RFSoC OFDM demonstration notebook, the OFDM symbol structure and mapping of pilots and null sub-carriers is based on the IEEE 802.11a/g (Wi-Fi) standard. In this notebook, we will restrict ourselves to the addition of null sub-carriers, since we will not demonstrate phase tracking and will use a special training symbol for channel estimation. The implementation of phase tracking can be seen in the RFSoC demonstration notebook.   

Let's set some important OFDM parameters: 

In [47]:
fs = 20e6 # Sampling rate 
N = 64 # No. of sub-carriers 

### 2.1 Training Symbol Generation <a class="anchor" id="training"></a>

As mentioned previously, channel estimation will be performed using special training symbol known to both transmitter and receiver. This will be based on the Legacy Long Training Field (L-LTF) used in the IEEE 802.11a/g standard. In the standard, the L-LTF consists of two training OFDM symbols (each with length $N$ = 64) preceded by a 32 sample CP. As a result, it is 160 samples long.  

The generation of the L-LTF is based on the following sequence:

In [48]:
#L-LTF sequence
LTFseq = np.array([0,0,0,0,0,0,1,1,-1,\
                  -1,1,1,-1,1,-1,1,1,1,\
                  1,1,1,-1,-1,1,1,-1,1,\
                  -1,1,1,1,1,0,1,-1,-1,\
                  1,1,-1,1,-1,1,-1,-1,\
                  -1,-1,-1,1,1,-1,-1,1,\
                  -1,1,-1,1,1,1,1,0,0,0,\
                  0,0])

In reference to [Figure 2.1](#fig-2.1), we have already completed the symbol generation and sub-carrier mapping stages. A total of 52 Binary Phase Shift Keying (BPSK) symbols have been generated (1,-1) and have been mapped to 52 OFDM sub-carriers. The remaining 12 sub-carriers are set to zero and these are the null sub-carriers. This includes the DC sub-carrier at the centre of the spectrum, 6 on the left edge of the frequency band and 5 on the right edge.   

We can now perform the IFFT to generate the training symbol:

In [49]:
LTFsymb = np.fft.ifft(np.fft.fftshift(LTFseq),N)

An FFT shift is performed to map the symbols to the correct IFFT bins. This is because the IFFT operates from 0 to $f_{s}$, whereas the sub-carriers are mapped assuming a frequency range $-f_{s}/2$ to $f_{s}/2$.  

Finally, to generate the L-LTF, we repeat the symbol twice and add a 32 sample CP. 

In [50]:
# Extract 32 sample CP 
LTFcp = LTFsymb[32:64]

# Concatenate to form L-LTF
LLTF = np.concatenate((LTFcp, LTFsymb, LTFsymb))

The L-LTF is used for channel estimation in this notebook and in the RFSoC OFDM demonstration. However, it can also be used for fine timing synchronisation, fine frequency synchronisation and integer frequency offset estimation in the receiver. In the IEEE 802.11a/g standard, another training symbol called the Legacy Short Training Field (L-STF) is also used. This will not be covered here. However, it is used for timing synchronisation and frequency offset estimation in the RFSoC OFDM demonstration system.   

### 2.2 OFDM Symbol Generation <a class="anchor" id="data"></a>

Having created the L-LTF, we will now go on to generate our data payload. This consists of a variable number of OFDM symbols carrying randomly generated (BPSK,QPSK,16-QAM) data symbols. We will add a CP of length $N/4 = 16$ samples to each OFDM symbol.
Lets generate a block of $N_{ofdm}N_{data}$ data symbols, where $N_{ofdm}$ is the number of OFDM symbols in the payload and $N_{data}$ is the number of data carrying sub-carriers:  

In [51]:
n_ofdm = 2 # No. of OFDM symbols
n_data = 52 # No. of data sub-carriers
nsym = n_ofdm * n_data # No. of data symbols
mod_scheme = 'QPSK' #Modulation scheme

# Generate data symbols
data = symbol_gen(nsym,mod_scheme)

We will now perform sub-carrier mapping and IFFT mdoualtion to obtain the OFDM symbols. The data symbols will be mapped in the same manner as the L-LTF sequence and there will be null sub-carriers at DC and on the band edges.

In [52]:
# Indices for data sub-carriers 
ind_1 = np.arange(start=6, stop=32)
ind_2 = np.arange(start=33, stop=59)
index = np.concatenate((ind_1, ind_2), axis=0)

# Initialisation of array to hold OFDM symbols 
ofdm_data = np.zeros(n_ofdm*N,np.complex64)
j = 0 
k = 0 

for i in range(n_ofdm):
    
    # Initialise array to hold data and null sub-carriers
    # (all null to begin with)
    sc_array = np.zeros(N,np.complex64)
    
    # Map data symbols to correct sub-carrier positions
    sc_array[index] = data[j:j+n_data] 
    
    # Perform IFFT modulation
    ofdm_data[k:k+N] = np.fft.ifft(np.fft.fftshift(sc_array),N) 
    
    # Increment
    j = j + n_data
    k = k + N

We will now add the 16 sample CP to the beginning of each OFDM symbol:

In [53]:
# Define function to add CP 
def add_cp(ofdm_symb,N,cp_len):
    
    #Extract CP
    cp = ofdm_symb[N-cp_len:N:1]
    
    # Concatenate CP and symbol 
    ofdm_symb_cp = np.concatenate((cp,ofdm_symb))
    
    return ofdm_symb_cp

cp_len = N // 4 # CP length is 1/4 of symbol period

# Add CP to each of the ofdm symbols 
ofdm_data_cp = np.zeros(n_ofdm*(N+cp_len),np.complex64)
j = 0
k = 0 

for i in range(n_ofdm):    
    ofdm_data_cp[k:(k+N+cp_len)] = add_cp(ofdm_data[j:j+N],N,cp_len)
    j = j + N  
    k = k + N + cp_len 

In order to create the final transmit signal, we attach the L-LTF at the beginning of the data payload. As such, the L-LTF is transmitted first:

In [54]:
# Concatenate L-LTF and data payload to form final transmit signal 
txSig = np.concatenate((LLTF,ofdm_data_cp))

## 3. Wireless Channel <a class="anchor" id="channel"></a>

At this stage, we will pass the OFDM signal through a baseband model of the wireless channel. The channel is comprised of a multipath filter and AWGN as illustrated below:  

<a class="anchor" id="fig-3.1"></a>
<figure>
<img src="images/channel copy.svg" style="width: 700px;"/> 
    <figcaption><b>Figure 3.1: Baseband Channel Model </b></figcaption>
</figure>

For the multipath channel filter, we will employ a standard 4-tap FIR filter with complex weights drawn from a zero mean normal distribution. This is the procedure used to simulate a Rayleigh Fading channel. With a 4 tap FIR, $d_{s}$ is equal to 3 sampling periods, meaning that the CP of length 16 is more than sufficient.

In [55]:
# Filter coefficients
ntap = 4
h = np.random.randn(ntap) + 1j*np.random.randn(ntap)

# Appy channel filter 
txSig_filt = np.convolve(txSig, h)

The AWGN is added with a power that is calculated based on a desired SNR: 

In [56]:
SNR = 35 # Desired SNR (dB) 
rxSig = awgn(txSig_filt,SNR)

## 4. OFDM Receiver <a class="anchor" id="ofdmrx"></a>

An illustration of the OFDM receiver steps is shown below: 


<a class="anchor" id="fig-4.1"></a>
<figure>
<img src="images/ofdm_rx copy.svg" style="width: 1000px;"/> 
    <figcaption><b>Figure 7.1: OFDM Receiver</b></figcaption>
</figure>

We will strart by extracting the L-LTF symbols and demodulate them using the FFT. These will be used for channel estimation purposes. In this example, we know the exact timing of the beginning of each symbol. However, in practice, this is unknown and an appropriate timing synchronisation algorithm must be used to acquire symbol timing.

The beginning of the first L-LTF OFDM symbol is sample 33 i.e. immediately after the 32 sample CP: 

In [57]:
# Extract received L-LTF OFDM symbols 
rx_LLTF_symb_1 = rxSig[33:97]
rx_LLTF_symb_2 = rxSig[97:161]

Let's now take the FFT of each of them to recover the transmitted sequences: 

In [58]:
LLTF_symb_1_demod = np.fft.fftshift(np.fft.fft(rx_LLTF_symb_1,N))
LLTF_symb_2_demod = np.fft.fftshift(np.fft.fft(rx_LLTF_symb_2,N))

Ok, we'll leave these for now and return to them when we do channel estimation. Now let's extract the data payload OFDM symbols and perform FFTs to recover the underlying data symbols. The CP is removed because it does not contain any information:

In [29]:
# Function to demodulate OFDM 
def ofdm_demod(ofdm_rx,N,cp_len):
    
    # Renove CP 
    ofdm_u = ofdm_rx[cp_len:(N+cp_len)]
    
    # Perform FFT 
    data = np.fft.fft(ofdm_u,N)
    
    return data

# Array to hold recovered  data symbols  
data_rx = np.zeros(n_ofdm*N,np.complex64)
j = 0
k = 0 

# Extract data payload (after end of L-LTF)
L = len(rxSig)
rxPayload = rxSig[160:L:1]

# Demodulate OFDM symbols in payload 
for i in range(n_ofdm):
    data_rx[j:j+N] = ofdm_demod(rxPayload[k:(k+N+cp_len)],N,cp_len)
    j = j + N
    k = k + N + cp_len 

### 4.1 Channel Estimation <a class="anchor" id="chanest"></a>

To be completed....